# SAFE_RL for Highway, Roundabout and Pong Environments

### Import Packages

In [1]:

import gymnasium as gym

import sys; sys.argv=['']; del sys

from stable_baselines3 import DQN,A2C,SAC,DDPG,PPO
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv
import random
import pandas as pd
from tkinter import Y
import numpy as np
from sklearn.utils import class_weight
from tqdm import trange
from torch.optim import Adam, SGD
import csv
import torch
import torch.nn as nn
from IPython.display import clear_output
# import matplotlib 
# matplotlib.use("Qt5Agg")
# from matplotlib import pyplot as plt
import highway_env
# from matplotlib import pyplot as plt
from scipy.signal import convolve, gaussian
from copy import deepcopy
import pandas as pd

import os
import io
import base64
import time
import glob
from IPython.display import HTML
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
from itertools import count
from time import time, strftime, localtime
import scipy.optimize
from tensorboardX import SummaryWriter
from core.models import *
from core.agent_ray_pd import AgentCollection
# import ray
# import envs
# from trpo import trpo
import pickle
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
import argparse
# from policy_distillation import main as policy_distillation 
# from student import Student
# from teacher import Teacher
from utils.utils import *
import util_highway as highway_pkg
import util_pong as pong_pkg
# from util_highway import create_img_dataset, save_student_model,CF_generation,load_student_model,load_teacher,\
#     student_model_play, compute_class_weight,check_validity, high_dimention_env_train,train_teacher, teacher_model_play
from src.star_gan.main import basemodels
from src.star_gan.data_loader import get_loader#,solver,model,main
import wandb

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("device: ",device)


2023-08-18 15:04:01.843895: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-18 15:04:01.894543: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


device:  cuda:0


### parameters for each models:
- set CF_method to SAFE_RL_attGAN(for SAFE-RL), Olson(for Olson model) and GANterfactual(for Huber model)
- set mode to test (for extracting metrics) or train 
- set  (d_lambda_cls, g_lambda_cls, lambda_rec_x, lambda_rec_sal, lambda_gp, lambda_counter, lambda_sal_fuse, g_loss_cls_of_d) to:
        (1,1,10,0,10,0,Ture) for GANterfactual model and (0,1,10,10,0,5,False) for SAFE_RL_attGAN model.

In [2]:
%tb
def get_PDCF_config():

    parser = argparse.ArgumentParser()

    parser.add_argument('--device', type=str, default=device,
                        help='set the device (cpu or cuda)')
    #environment 
    parser.add_argument('--env', type=str, default='roundabout', help='highway, roundabout')
    parser.add_argument('--env_name', type=str, default='roundabout-v0', help='highway-fast-v0, roundabout-v0')
    parser.add_argument('--num_Of_agents', type=int, default=6, help='1 for pong, num_Of_vehicles_Under_vision for highway which is 6')
    parser.add_argument('--num_Of_attributes', type=int, default=4, help='4 for pong, vehicle_attr for highway which is 4')
    parser.add_argument('--num_of_actions', type=int, default=5, help='6 for pong, 5 for highway')
    parser.add_argument('--num_of_stack', type=int, default=4, help='observation history size')
    parser.add_argument('--obs_size', type=int, default= (448, 448), help='(84,84) for pong, (512, 128) for highway, (448,448) for roundabout')
    parser.add_argument('--image_size', type=int, default=(448, 448), help='(210,160) for pong, (512, 128) for highway, (448,448) for roundabout')
    parser.add_argument('--scaling', type=int, default=5, help='5 for highway, 5 for roundabout')
    parser.add_argument('--centering_position', type=int, default=[0.5, 0.85], help='[0.2, 0.5] for highway, [0.5, 0.85] for roundabout')
    parser.add_argument('--batch_size', type=int, default=8, help='mini-batch size')

    # Network, env, MDP, seed
    parser.add_argument('--hidden-size', type=int, default=256,
                        help='number of hidden units per layer')  
    parser.add_argument('--env_input_size', type=int, 
                        default=parser.parse_args().num_Of_attributes, #+ num_Of_agents*num_of_actions,
                        # default=parser.parse_args().num_Of_agents*parser.parse_args().num_Of_attributes, #+ num_Of_agents*num_of_actions,
                            help='number of hidden units per layer')
    parser.add_argument('--env_output_size', type=int, 
                        default=parser.parse_args().num_of_actions,
                            help='number of hidden units per layer')
    parser.add_argument('--num-layers', type=int, default=2,
                        help='number of hidden layers')
    parser.add_argument('--gamma', type=float, default=0.995, metavar='G',
                        help='discount factor (default: 0.995)')
    parser.add_argument('--tau', type=float, default=0.97, metavar='G',
                        help='gae (default: 0.97)')
    parser.add_argument('--seed', type=int, default=1, metavar='N',
                        help='random seed (default: 1)')
    parser.add_argument('--load-models', default=True, action='store_true',
                        help='load_pretrained_models')

    # Teacher policy training
    parser.add_argument('--teacher_alg', type=str, default='PPO', help='PPO, ')
    parser.add_argument('--teacher_models_dir', type=str, default='teacher_models/',)
    parser.add_argument('--agent-count', type=int, default=10, metavar='N',
                        help='number of agents (default: 100)')
    parser.add_argument('--num-teachers', type=int, default=1, metavar='N',
                        help='number of teacher policies (default: 1)')
    parser.add_argument('--max-kl', type=float, default=1e-2, metavar='G',
                        help='max kl value (default: 1e-2)')
    parser.add_argument('--cg-damping', type=float, default=1e-2, metavar='G',
                        help='damping for conjugate gradient (default: 1e-2)')
    parser.add_argument('--cg-iter', type=int, default=10, metavar='G',
                        help='maximum iteration of conjugate gradient (default: 1e-1)')
    parser.add_argument('--l2-reg', type=float, default=1e-3, metavar='G',
                        help='l2 regularization parameter for critics (default: 1e-3)')
    parser.add_argument('--teacher-batch-size', type=int, default=1000, metavar='N',
                        help='per-iteration batch size for each agent (default: 1000)')
    parser.add_argument('--sample-batch-size', type=int, default=10000, metavar='N',
                        help='expert batch size for each teacher (default: 10000)')
    parser.add_argument('--render', action='store_true',
                        help='render the environment')
    parser.add_argument('--log-interval', type=int, default=1, metavar='N',
                        help='interval between training status logs (default: 10)')
    parser.add_argument('--num-workers', type=int, default=60,
                        help='number of workers for parallel computing')
    parser.add_argument('--num-teacher-episodes', type=int, default=10, metavar='N',
                        help='num of teacher training episodes (default: 100)')

    # Student policy training
    parser.add_argument('--student_models_dir', type=str, default='student_models/',)
    parser.add_argument('--lr', type=float, default=1e-3, metavar='G',
                        help='adam learnig rate (default: 1e-3)')
    parser.add_argument('--test-interval', type=int, default=10, metavar='N',
                        help='interval between training status logs (default: 10)')
    parser.add_argument('--student-batch-size', type=int, default=128, metavar='N',
                        help='per-iteration batch size for student (default: 1000)')
    parser.add_argument('--batch-type', type=str, default='random',
                        help='batch should be selected random or respectively')
    parser.add_argument('--sample-interval', type=int, default=10, metavar='N',
                        help='frequency to update expert data (default: 10)')
    parser.add_argument('--testing-batch-size', type=int, default=512, metavar='N',
                        help='batch size for testing student policy (default: 10000)')
    parser.add_argument('--num-student-episodes', type=int, default=5, metavar='N',
                        help='num of student training episodes (default: 1000)')
    parser.add_argument('--report_step', type=int, default=500, metavar='N')
    parser.add_argument('--loss-metric', type=str, default='kl',
                        help='metric to build student objective, nll, wasserstein or kl or kl_cross or probabilistic')
    parser.add_argument('--temperature', type=float, default=0.9,
                        help='KL temperature')
    parser.add_argument('--betta', type=float, default=0.7,
                        help='beta*kl loss + (1-betta)*cross loss')
    parser.add_argument('--algo', type=str, default='sgd',
                        help='update method')
    parser.add_argument('--storm-interval', type=int, default=10, metavar='N',
                        help='frequency of storm (default: 10)')
    parser.add_argument('--init-alpha', type=float, default=1.0, metavar='G',
                        help='storm init alpha (default: 1.0)')
    parser.add_argument('--optimizer', type=str, default='ADAM', metavar='G',
                        help='student network otimizer, SGD or ADAM')
    # counterfactual setting
    parser.add_argument('--CF_method', type=str, default='SAFE_RL_attGAN', help='PDCF, SAFE_RL_starGAN, SAFE_RL_attGAN, Olson, GANterfactual')
    parser.add_argument('--dataset_dir', type=str, default="dataset", help='saving image of environments for olson and GANterfactual models')
    parser.add_argument('--mode', type=str, default="train", help='train, test')
    parser.add_argument('--saliency_method', type=str, default="EigenCAM", \
        help='HiResCAM, AblationCAM, ScoreCAM, LayerCAM, FullGradDeep, FeatureFactorization, EigenCAM')
    parser.add_argument('--saliency_dim', type=int, default=1, \
        help='1 EigenCAM, c_dim for GradCAM')
    parser.add_argument('--wandb_log', type=bool, default=False, help='Weight and bias')

    args = parser.parse_args("")
    return args 

def get_Olson_config(PDCF_config):
    print('Parsing arguments')
    parser = argparse.ArgumentParser()
    parser.add_argument('--batch_size', type=int, default=PDCF_config.batch_size)
    parser.add_argument('--epsilon', type=float, default=.2)
    parser.add_argument('--lr', type=float, default=15e-4)
    # Output directory of the model that creates counterfactual states
    parser.add_argument('--checkpoint_dir', type=str, default=\
        os.path.join("baseline", PDCF_config.CF_method, PDCF_config.env, PDCF_config.teacher_alg, "models"))
    parser.add_argument('--epochs', type=int, default=300)

    parser.add_argument('--latent', type=int, default=16)#16 #64
    parser.add_argument('--wae_latent', type=int, default=128)
    parser.add_argument('--agent_latent', type=int, default=512)
    parser.add_argument('--seed', type=int, default=13)
    parser.add_argument('--env', type=str, default=PDCF_config.env)
    # Directory of the used wasserstein encoder
    parser.add_argument('--Q', type=str, default="../../res/models/PacMan_FearGhost2_3_Olson_wae/Q")
    parser.add_argument('--P', type=str, default="../../res/models/PacMan_FearGhost2_3_Olson_wae/P")
    parser.add_argument('--missing', type=str, default="none")
    # Directory of the RL-Agent that should be explained
    parser.add_argument('--agent_file', type=str, default="../../res/agents/ACER_PacMan_FearGhost2_cropped_5actions_40M_3.pt")
    parser.add_argument('--enc_lam', type=float, default=5)
    parser.add_argument('--clip', type=float, default=.0001)
    parser.add_argument('--gen_lam', type=float, default=.5)
    parser.add_argument('--starting_epoch', type=int, default=0)
    parser.add_argument('--info', type=str, default="")
    parser.add_argument('--cf_loss', type=str, default="None")
    parser.add_argument('--m_frames', type=int, default=40)
    parser.add_argument('--fskip', type=int, default=4)
    parser.add_argument('--use_agent', type=int, default=1)
    parser.add_argument('--gpu', type=int, default=0)

    parser.add_argument('--use_dataset', type=bool, default=True)
    # The dataset used for training the model
    parser.add_argument('--dataset_dir', type=str, default="../../res/datasets/ACER_PacMan_FearGhost2_cropped_5actions_40M_3_Unique")
    parser.add_argument('--img_size', type=str, default=PDCF_config.image_size)
    parser.add_argument('--img_channels', type=int, default=3)
    parser.add_argument('--action_size', type=int, default=PDCF_config.num_of_actions)
    parser.add_argument('--is_pacman', type=bool, default=True)
    # Directories.
    parser.add_argument('--log_dir', type=str, default=os.path.join("baseline", PDCF_config.CF_method, PDCF_config.env, PDCF_config.teacher_alg, "logs"))
    parser.add_argument('--model_save_dir', type=str, default=os.path.join("baseline", PDCF_config.CF_method, PDCF_config.env, PDCF_config.teacher_alg, "models"))
    parser.add_argument('--sample_dir', type=str, default=os.path.join("baseline", PDCF_config.CF_method, PDCF_config.env, PDCF_config.teacher_alg, "samples"))
    parser.add_argument('--result_dir', type=str, default=os.path.join("baseline", PDCF_config.CF_method, PDCF_config.env, PDCF_config.teacher_alg, "results"))
    parser.add_argument('--device', type=str, default=PDCF_config.device)
    parser.add_argument('--wandb_log', type=bool, default=PDCF_config.wandb_log, help='Weight and bias')
    parser.add_argument('--agent_algo', type=str, default=PDCF_config.teacher_alg, help='DQN, PPO, A2C')
    parser.add_argument('--mode', type=str, default=PDCF_config.mode, help='train, test')
    parser.add_argument('--metrics', default=["KID", "FID", "LPIPS", "IS", "sparsity", "mean_dis", "validity"], help='test model from this step')
    parser.add_argument('--load_epoch', type=int, default=None)#train from scratch: None, load best for test: 39, train from the best last experiment: 20


    args = parser.parse_args()
    return args

def get_starGAN_config(PDCF_config):
    parser = argparse.ArgumentParser()

    # Model configuration.
    parser.add_argument('--c_dim', type=int, default=PDCF_config.num_of_actions, help='dimension of domain labels (1st dataset)')
    parser.add_argument('--c2_dim', type=int, default=PDCF_config.num_of_actions, help='dimension of domain labels (2nd dataset)')
    parser.add_argument('--celeba_crop_size', type=int, default=178, help='crop size for the CelebA dataset')
    parser.add_argument('--rafd_crop_size', type=int, default=256, help='crop size for the RaFD dataset')
    parser.add_argument('--image_size', type=int, default=PDCF_config.image_size[0], help='image resolution')
    parser.add_argument('--g_conv_dim', type=int, default=64, help='number of conv filters in the first layer of G')
    parser.add_argument('--d_conv_dim', type=int, default=64, help='number of conv filters in the first layer of D')
    parser.add_argument('--g_repeat_num', type=int, default=6, help='number of residual blocks in G')
    parser.add_argument('--d_repeat_num', type=int, default=6, help='number of strided conv layers in D')
    parser.add_argument('--d_lambda_cls', type=float, default=1, help='weight for domain classification loss')
    parser.add_argument('--g_lambda_cls', type=float, default=1, help='weight for domain classification loss')
    parser.add_argument('--lambda_rec_x', type=float, default=10, help='weight for reconstruction loss of image and obs')
    parser.add_argument('--lambda_rec_sal', type=float, default=10, help='weight for reconstruction loss of saliency map')
    parser.add_argument('--lambda_gp', type=float, default=10, help='weight for gradient penalty')
    parser.add_argument('--lambda_counter', type=float, default=0, help='weight for counter loss')
    parser.add_argument('--lambda_sal_fuse', type=float, default=5, help='weight for counter loss')
    
    parser.add_argument('--g_loss_cls_of_d', type=bool, default=False, help='')

    # Training configuration.
    parser.add_argument('--dataset', type=str, default=PDCF_config.env, choices=['CelebA', 'RaFD', 'Both'])
    parser.add_argument('--batch_size', type=int, default=PDCF_config.batch_size, help='mini-batch size')
    parser.add_argument('--num_iters', type=int, default=400000, help='number of total iterations for training D')
    parser.add_argument('--num_iters_decay', type=int, default=100000, help='number of iterations for decaying lr')
    parser.add_argument('--g_lr', type=float, default=0.0001, help='learning rate for G')
    parser.add_argument('--d_lr', type=float, default=0.0001, help='learning rate for D')
    parser.add_argument('--n_critic', type=int, default=5, help='number of D updates per each G update')
    parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for Adam optimizer')
    parser.add_argument('--beta2', type=float, default=0.999, help='beta2 for Adam optimizer')
    parser.add_argument('--resume_iters', type=int, default=None, help='resume training from this step')
    
    # Test configuration.
    parser.add_argument('--test_iters', type=int, default=200000, help='test model from this step')
    parser.add_argument('--metrics', default=["KID", "FID", "LPIPS", "IS", "sparsity", "mean_dis", "validity"], help='test model from this step')

    # Miscellaneous.
    parser.add_argument('--num_workers', type=int, default=PDCF_config.num_workers)
    parser.add_argument('--mode', type=str, default=PDCF_config.mode)
    # Directories.
    parser.add_argument('--log_dir', type=str, default=os.path.join("baseline", PDCF_config.CF_method, PDCF_config.env, PDCF_config.teacher_alg, "logs"))
    parser.add_argument('--model_save_dir', type=str, default=os.path.join("baseline", PDCF_config.CF_method, PDCF_config.env, PDCF_config.teacher_alg, "models"))
    parser.add_argument('--sample_dir', type=str, default=os.path.join("baseline", PDCF_config.CF_method, PDCF_config.env, PDCF_config.teacher_alg, "samples"))
    parser.add_argument('--result_dir', type=str, default=os.path.join("baseline", PDCF_config.CF_method, PDCF_config.env, PDCF_config.teacher_alg, "results"))

    # Step size.
    parser.add_argument('--log_step', type=int, default=10)
    parser.add_argument('--sample_step', type=int, default=1000)
    parser.add_argument('--model_save_step', type=int, default=20000)
    parser.add_argument('--lr_update_step', type=int, default=1000)

    # Extensions
    parser.add_argument('--image_channels', type=int, default=3, help='number of channels of each image')
    parser.add_argument('--agent_path', type=str, default=None, help='path to a h5 file containing a rl agent')
    parser.add_argument('--counter_mode', type=str, default='cross_entropy', help='whether to use "raw","cross_entropy", "softmax",'
                                                                              '"advantage", or "z-score"')
    parser.add_argument('--selective_counter', type=bool, default=False, help='whether to only use samples where'
                                                                             'c_trg != c_org for counter-loss')
    # if PDCF_config.CF_method == "GANterfactual":
    #     parser.add_argument('--agent_type', type=str, default="acer", help='which agent type to use (deepq,'
    #                                                                             'olson, acer)')
    # elif PDCF_config.CF_method == "Olson":
    #     parser.add_argument('--agent_type', type=str, default="olson", help='which agent type to use (deepq,'
    #                                                                             'olson, acer)')
    # else:
    parser.add_argument('--agent_type', type=str, default=PDCF_config.CF_method, help='which agent type to use (deepq,'
                                                                                'olson, acer)')
        
    parser.add_argument('--ablate_agent', type=bool, default=False, help='whether to ablate the laser canon before'
                                                                         'inputting a frame to the agent')
    parser.add_argument('--agent_algo', type=str, default=PDCF_config.teacher_alg, help='DQN, PPO, A2C')
    parser.add_argument('--saliency_method', type=str, default=PDCF_config.saliency_method, \
        help='HiResCAM, AblationCAM, ScoreCAM, LayerCAM, FullGradDeep, FeatureFactorization')
    args = parser.parse_args("")
    return args

args = get_PDCF_config()

No traceback available to show.


### main loop
- put the environments that need to be run in the first for loop, e,g. ["highway", "pong"]
- put the DRL models on the second for loop, e,g. ["DQN", "PPO"]
- for the first run only uncomment the "train_teacher" function to create the needed DRL.
- for the first run only uncomment the "create_img_dataset" function to create the needed datasets.

In [3]:

for env_name in ["pong", "highway", "roundabout"]:
    if env_name == "highway":
        args.env ='highway'
        args.env_name = 'highway-fast-v0'
        args.obs_size = (128, 512) 
        args.image_size = (128, 512) 
        args.centering_position = [0.2, 0.5] 
    elif env_name == "roundabout":
        args.env ='roundabout'
        args.env_name = 'roundabout-v0'
        args.obs_size = (448,448)
        args.image_size = (448,448)
        args.centering_position = [0.5, 0.85]
    elif env_name == "pong":
        args.env ='pong'
        args.env_name = 'PongNoFrameskip-v4'
        args.obs_size = (84,84)
        args.image_size = (84,84)
        args.centering_position = [0.5, 0.85]
        args.num_Of_attributes=4
        args.num_of_actions=6
    
    for DRL_agent in ["A2C", "PPO", "DQN"]: #"PPO", 'DQN', 'A2C',
        args.teacher_alg = DRL_agent
        if env_name in ["pong"]:
            # teacher_model = pong_pkg.train_teacher(args)
            teacher_model = pong_pkg.load_teacher(args)
            # pong_pkg.create_img_dataset(args, teacher_model)
        else:
            # teacher_model = highway_pkg.train_teacher(args)
            teacher_model = highway_pkg.load_teacher(args)
            highway_pkg.teacher_model_play(teacher_model=teacher_model, config=args)
            # highway_pkg.create_img_dataset(args, teacher_model)
    
        if(args.wandb_log):
            starGAN_config = get_starGAN_config(PDCF_config=args)
            wandb.init(
            # set the wandb project where this run will be logged
            project="SAFE_RL",
            entity='amirsamadi',
            # track hyperparameters and run metadata
            config={
            "PDCF_config": args,
            "starGAN_config": starGAN_config,})

        if(args.CF_method in ["SAFE_RL_starGAN", "SAFE_RL_attGAN"]):
            starGAN_config = get_starGAN_config(PDCF_config=args) 
            data_loader = get_loader(PDCF_config=args, starGAN_config=starGAN_config, mode=args.mode)
            basemodels(starGAN_config=starGAN_config, PDCF_config=args, data_loader=data_loader, teacher_model=teacher_model)

        elif(args.CF_method=="Olson"):
            Olson_config = get_Olson_config(PDCF_config=args)
            starGAN_config = get_starGAN_config(PDCF_config=args)
            data_loader = get_loader(PDCF_config=args, starGAN_config=starGAN_config, mode=args.mode)
            basemodels(starGAN_config=starGAN_config, PDCF_config=args, data_loader=data_loader, teacher_model=teacher_model, Olson_config=Olson_config)
        
        elif(args.CF_method=="GANterfactual"):
            starGAN_config = get_starGAN_config(PDCF_config=args)
            data_loader = get_loader(PDCF_config=args, starGAN_config=starGAN_config, mode=args.mode)
            basemodels(starGAN_config=starGAN_config, PDCF_config=args, data_loader=data_loader, teacher_model=teacher_model)
        
        else:
            raise("this CF method is not implemented yet!")


/home/samadi_a@WMGDS.WMG.WARWICK.AC.UK/anaconda3/envs/PDCF1/lib/python3.8/site-packages/stable_baselines3/common/save_util.py:166: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: an integer is required (got type bytes)
  warnings.warn(
/home/samadi_a@WMGDS.WMG.WARWICK.AC.UK/anaconda3/envs/PDCF1/lib/python3.8/site-packages/stable_baselines3/common/vec_env/patch_gym.py:95: UserWarning: You loaded a model that was trained using OpenAI Gym. We strongly recommend transitioning to Gymnasium by saving that model again.
  warnings.warn(
/home/samadi_a@WMGDS.WMG.WARWICK.AC.UK/anaconda3/envs/PDCF1/lib/python3.8/site-packages/stable_baselines3/common/base_class.py:752: UserWarning: You are probably loading a model saved with SB3 < 1.7.0, we deactivated exact_match so you can save the model again to avoid issues in the future (see https://github.com/DLR-RM/stable-baselines3/issues/1233 for more info). Original error

number of samples in dataset:35784
Namespace(ablate_agent=False, agent_algo='A2C', agent_path=None, agent_type='SAFE_RL_attGAN', batch_size=8, beta1=0.5, beta2=0.999, c2_dim=6, c_dim=6, celeba_crop_size=178, counter_mode='cross_entropy', d_conv_dim=64, d_lambda_cls=1, d_lr=0.0001, d_repeat_num=6, dataset='pong', g_conv_dim=64, g_lambda_cls=1, g_loss_cls_of_d=False, g_lr=0.0001, g_repeat_num=6, image_channels=3, image_size=84, lambda_counter=0, lambda_gp=10, lambda_rec_sal=10, lambda_rec_x=10, lambda_sal_fuse=5, log_dir='baseline/SAFE_RL_attGAN/pong/A2C/logs', log_step=10, lr_update_step=1000, metrics=['KID', 'FID', 'LPIPS', 'IS', 'sparsity', 'mean_dis', 'validity'], mode='train', model_save_dir='baseline/SAFE_RL_attGAN/pong/A2C/models', model_save_step=20000, n_critic=5, num_iters=400000, num_iters_decay=100000, num_workers=60, rafd_crop_size=256, result_dir='baseline/SAFE_RL_attGAN/pong/A2C/results', resume_iters=None, saliency_method='EigenCAM', sample_dir='baseline/SAFE_RL_attGAN/po

KeyboardInterrupt: 